In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2022-03-01'
final_date = '2022-03-31'
query_error_year = 2022
query_error_month = 3

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2022\03. MARZO\DATA\ERROR MARZO 2022.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 10)
#                 & (query_error['year'] == 2021)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 10 )
#             & (query_error['year'] == 2021), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15 22:01:46,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15 22:02:08,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01 19:53:09,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,NaN
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12 17:07:39,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12 17:07:25,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN


In [4]:
query_error.shape

(76892, 17)

In [5]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int64
year                             int64
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int64
mes_cierre                      object
dtype: object

In [6]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [7]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [8]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_2552\3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
65839,65839,a890a2d0-14d5-4e57-a0cd-0a8616c3dd76,NEPTUNO,SPQYRVHQ5,2022-03-01,128.41,ERROR,Bank account information not available,2022-03-01 08:19:35,3,2022,NEW,SIN_CLABE,a890a2d0-14d5-4e57-a0cd-0a8616c3dd76,2022-03-01,1,Actual
65314,65314,9eca2c9e-7586-4416-936f-75a768faff41,Cremeria Lety,SBWXML7F8,2022-02-27,64.20,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-03-01 17:37:30,3,2022,NEW,ERROR_USUAL,9eca2c9e-7586-4416-936f-75a768faff41,2022-03-01,1,Actual
65319,65319,3eb75ab3-5471-42d9-b72b-0829d2073bf5,eltrebol dulcerias,MZ367L2W7,2022-02-27,571.57,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2022-03-01 17:37:23,3,2022,NEW,ERROR_USUAL,3eb75ab3-5471-42d9-b72b-0829d2073bf5,2022-03-01,1,Actual
65321,65321,2dbdd79f-d96d-4a1d-bcdd-c4bf947eb483,Barbacoa Cathy,M78YS9K3X,2022-02-27,292.27,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2022-03-01 17:37:26,3,2022,NEW,ERROR_USUAL,2dbdd79f-d96d-4a1d-bcdd-c4bf947eb483,2022-03-01,1,Actual
65323,65323,f71d063f-0442-46ce-ac47-76cf1ea70cd4,MENU FELIZ,NT6RSVPSQ,2022-02-27,124.57,ERROR,DEV.SPEIFALTA INFORMAC PARA CO,2022-03-01 17:37:24,3,2022,NEW,ERROR_USUAL,f71d063f-0442-46ce-ac47-76cf1ea70cd4,2022-03-01,1,Actual


In [9]:
query_error_current_month.shape

(11866, 17)

In [10]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [11]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [12]:
duplicates_query_error.shape

(0, 17)

# PAGOS NEW DE MESES ANTERIORES

In [13]:
query_new_last_month = query_error[(query_error['mes_cierre'] != 'Actual') & (query_error['status_cubeta'] == 'NEW')]
query_new_last_month.sort_values(by=['day'],inplace=True)
query_new_last_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_2552\3381233170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_new_last_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
48686,48686,ae868353-ad64-43b7-aeb2-8a2058b123bf,Calzados Viany,58DSB8D6L,2021-12-07,1.80,ERROR,Tipo de operacion erronea,2022-02-01 13:23:55,2,2022,NEW,ERROR_USUAL,ae868353-ad64-43b7-aeb2-8a2058b123bf,2022-02-01,1,NaN


In [14]:
query_new_last_month.shape

(1, 17)

# INFORMACIÓN QUERIES - PAGOS

In [15]:
name_header = ['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','status','bank_name'
               ,'payment_date_local','bank_account_owner_name','bank_account_number','swap']

### NEXT MONTH

In [16]:
# TRADICIONAL
# trad_files_csv_2021_may = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\06. JUNE\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
# fp_files_csv_2021_may = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\06. JUNE\QUERIES\*.csv', recursive=False)

In [17]:
# disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_may])
# disp_files_csv_2021_may['date_query'] = disp_files_csv_2021_may["file_name"].str[-14:-4]
# disp_files_csv_2021_may['date_query']= pd.to_datetime(disp_files_csv_2021_may['date_query'])

In [18]:
# fp_disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_may])
# fp_disp_files_csv_2021_may['date_query'] = fp_disp_files_csv_2021_may["file_name"].str[-14:-4]
# fp_disp_files_csv_2021_may['date_query']= pd.to_datetime(fp_disp_files_csv_2021_may['date_query']) 

### CURRENT MONTH

In [19]:
# TRADICIONAL
trad_files_csv_2022_current = glob.glob(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\03. MARCH\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2022_current = glob.glob(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\03. MARZO\QUERIES\*.csv', recursive=False)

In [20]:
disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2022_current])
disp_files_csv_2022_current['date_query'] = disp_files_csv_2022_current["file_name"].str[-14:-4]
disp_files_csv_2022_current['date_query']= pd.to_datetime(disp_files_csv_2022_current['date_query'])

In [21]:
fp_disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2022_current])
fp_disp_files_csv_2022_current['date_query'] = fp_disp_files_csv_2022_current["file_name"].str[-14:-4]
fp_disp_files_csv_2022_current['date_query']= pd.to_datetime(fp_disp_files_csv_2022_current['date_query']) 

### LAST MONTH

In [22]:
# TRADICIONAL
trad_files_csv_2021_last = glob.glob(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\02. FEBRUARY\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_last = glob.glob(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\02. FEBRERO\QUERIES\*.csv', recursive=False)

In [23]:
disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_last])
disp_files_csv_2021_last['date_query'] = disp_files_csv_2021_last["file_name"].str[-14:-4]
disp_files_csv_2021_last['date_query']= pd.to_datetime(disp_files_csv_2021_last['date_query'])

In [24]:
fp_disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_last])
fp_disp_files_csv_2021_last['date_query'] = fp_disp_files_csv_2021_last["file_name"].str[-14:-4]
fp_disp_files_csv_2021_last['date_query']= pd.to_datetime(fp_disp_files_csv_2021_last['date_query']) 

## ALL FILES

In [25]:
allfiles = pd.concat([disp_files_csv_2022_current,fp_disp_files_csv_2022_current
                     ,disp_files_csv_2021_last,fp_disp_files_csv_2021_last])
allfiles.reset_index()
allfiles.fillna('BLANK',inplace=True)
allfiles['date_query']= pd.to_datetime(allfiles['date_query'])
allfiles['report_date_local']= pd.to_datetime(allfiles['report_date_local'])
allfiles.loc[(allfiles['file_name'].str.contains('FP'),'tipo_dispersion')] = 'FP'
allfiles.loc[(allfiles['file_name'].str.contains('SWAP'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('BNTE'),'tipo_dispersion')] = 'TRAD'
allfiles.sort_values(by=['date_query','report_id'],inplace=True)
allfiles.drop(['status'],axis=1,inplace=True)
allfiles.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,swap,file_name,disbursement_method,date_query,tipo_dispersion
19771,44c18ed3-a539-4b92-a820-b8bfe240a2ec,Butic Dani,guillerminatrejoponce@gmail.com,2226SCJM5,2022-01-18,3.20,"Banco Nacional de México, S.A.",2022-02-01 08:16:00,Guillermina Trejo Ponce,'002180904725962832,2022-02-01 08:06:48,"Guillermina Trejo Ponce,,3.20,0021809047259628...",SWAP 2022-02-01.csv,BLANK,2022-02-01,TRAD
67960,eca46980-db16-48bb-9524-2590bffe7abc,Delicarne,juanclusa@gmail.com,2228H2KBV,2022-02-01,284.09,"Banco Azteca, S.A.",2022-02-01 08:15:30,Ursula Pacheco Gomez,'127180013357157409,2022-02-01 08:14:20,"Ursula Pacheco Gomez ,,284.09,1271800133571574...",SWAP 2022-02-01.csv,BLANK,2022-02-01,TRAD
48135,a7b19a9a-e103-415e-95a7-e8b2d13a1b0e,Pintura Y Barnices La Brocha,noelrosalesuscanga@hotmail.com,2228YHDPP,2022-02-01,"2,029.10","HSBC México, S.A.",2022-02-01 08:14:58,Hilda luisa Uscanga Hernández,'021847040592603722,2022-02-01 08:11:23,"Hilda luisa Uscanga Hernández ,,2029.10,021847...",SWAP 2022-02-01.csv,BLANK,2022-02-01,TRAD
16315,38d51729-0d41-4123-9918-ae8d4d65432f,La Campesina,la_campesina2020@outlook.es,222G5TLRQ,2022-02-01,302.14,BBVA BANCOMER,2022-02-01 08:14:58,Jose Luis Contreras Armenta,'012221001867460390,2022-02-01 08:06:19,"Jose Luis Contreras Armenta,,302.14,0122210018...",SWAP 2022-02-01.csv,BLANK,2022-02-01,TRAD
16766,3a67c0d6-90ef-4d85-a491-048f02783655,Farmacia Marella,dianarm94@hotmail.com,222PPYCK4,2022-02-01,"2,006.07","Banco Azteca, S.A.",2022-02-01 08:16:32,Diana Laura Ramirez Martinez,'127622013434344716,2022-02-01 08:06:23,"Diana Laura Ramirez Martinez,,2006.07,12762201...",SWAP 2022-02-01.csv,BLANK,2022-02-01,TRAD


In [26]:
def tipo_pago(allfiles):
    if allfiles['date_query'] == allfiles['report_date_local']:
        return 'Pago del día'
    else:
        return 'Reprogramación'
allfiles['tipo_pago'] = allfiles.apply(tipo_pago, axis=1)

In [27]:
allfiles.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,swap,file_name,disbursement_method,date_query,tipo_dispersion,tipo_pago
19771,44c18ed3-a539-4b92-a820-b8bfe240a2ec,Butic Dani,guillerminatrejoponce@gmail.com,2226SCJM5,2022-01-18,3.20,"Banco Nacional de México, S.A.",2022-02-01 08:16:00,Guillermina Trejo Ponce,'002180904725962832,2022-02-01 08:06:48,"Guillermina Trejo Ponce,,3.20,0021809047259628...",SWAP 2022-02-01.csv,BLANK,2022-02-01,TRAD,Reprogramación
67960,eca46980-db16-48bb-9524-2590bffe7abc,Delicarne,juanclusa@gmail.com,2228H2KBV,2022-02-01,284.09,"Banco Azteca, S.A.",2022-02-01 08:15:30,Ursula Pacheco Gomez,'127180013357157409,2022-02-01 08:14:20,"Ursula Pacheco Gomez ,,284.09,1271800133571574...",SWAP 2022-02-01.csv,BLANK,2022-02-01,TRAD,Pago del día
48135,a7b19a9a-e103-415e-95a7-e8b2d13a1b0e,Pintura Y Barnices La Brocha,noelrosalesuscanga@hotmail.com,2228YHDPP,2022-02-01,"2,029.10","HSBC México, S.A.",2022-02-01 08:14:58,Hilda luisa Uscanga Hernández,'021847040592603722,2022-02-01 08:11:23,"Hilda luisa Uscanga Hernández ,,2029.10,021847...",SWAP 2022-02-01.csv,BLANK,2022-02-01,TRAD,Pago del día
16315,38d51729-0d41-4123-9918-ae8d4d65432f,La Campesina,la_campesina2020@outlook.es,222G5TLRQ,2022-02-01,302.14,BBVA BANCOMER,2022-02-01 08:14:58,Jose Luis Contreras Armenta,'012221001867460390,2022-02-01 08:06:19,"Jose Luis Contreras Armenta,,302.14,0122210018...",SWAP 2022-02-01.csv,BLANK,2022-02-01,TRAD,Pago del día
16766,3a67c0d6-90ef-4d85-a491-048f02783655,Farmacia Marella,dianarm94@hotmail.com,222PPYCK4,2022-02-01,"2,006.07","Banco Azteca, S.A.",2022-02-01 08:16:32,Diana Laura Ramirez Martinez,'127622013434344716,2022-02-01 08:06:23,"Diana Laura Ramirez Martinez,,2006.07,12762201...",SWAP 2022-02-01.csv,BLANK,2022-02-01,TRAD,Pago del día


In [28]:
allfiles.shape

(4949950, 17)

# CRUCE 1 - MES ACTUAL

In [29]:
cruce_1 = allfiles.merge(query_error_current_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status'
                                                   ,'error_description','esquema_query','status_cubeta']],on='report_id',how='left')
cruce_1['month'].fillna('Not found',inplace=True)
cruce_1.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,44c18ed3-a539-4b92-a820-b8bfe240a2ec,Butic Dani,guillerminatrejoponce@gmail.com,2226SCJM5,2022-01-18,3.20,"Banco Nacional de México, S.A.",2022-02-01 08:16:00,Guillermina Trejo Ponce,'002180904725962832,...,TRAD,Reprogramación,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,eca46980-db16-48bb-9524-2590bffe7abc,Delicarne,juanclusa@gmail.com,2228H2KBV,2022-02-01,284.09,"Banco Azteca, S.A.",2022-02-01 08:15:30,Ursula Pacheco Gomez,'127180013357157409,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,a7b19a9a-e103-415e-95a7-e8b2d13a1b0e,Pintura Y Barnices La Brocha,noelrosalesuscanga@hotmail.com,2228YHDPP,2022-02-01,"2,029.10","HSBC México, S.A.",2022-02-01 08:14:58,Hilda luisa Uscanga Hernández,'021847040592603722,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,38d51729-0d41-4123-9918-ae8d4d65432f,La Campesina,la_campesina2020@outlook.es,222G5TLRQ,2022-02-01,302.14,BBVA BANCOMER,2022-02-01 08:14:58,Jose Luis Contreras Armenta,'012221001867460390,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,3a67c0d6-90ef-4d85-a491-048f02783655,Farmacia Marella,dianarm94@hotmail.com,222PPYCK4,2022-02-01,"2,006.07","Banco Azteca, S.A.",2022-02-01 08:16:32,Diana Laura Ramirez Martinez,'127622013434344716,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [30]:
cruce_1.shape

(4949950, 25)

In [31]:
cruce_1_final = cruce_1[(cruce_1['month'] != 'Not found')]
cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_1_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_2552\1938979547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
2048002,5fa39939-1cba-463c-83f5-0e9f5b94c973,Boutique Infantil Chamacos,rscp_86@hotmail.com,24W55KF8W,2022-02-26,306.64,SANTANDER,2022-02-26 10:58:49,Rocio Sarai Cortes Peña,'014180566459550548,...,TRAD,Pago del día,306.64,2022-03-01 17:37:14,2022-03-01,3.00,ERROR,DEV.SPEICUENTA BLOQUEADA,ERROR_USUAL,NEW
2048133,0e868049-3ea7-4366-87d4-765e417a07c1,Farmacia Esperanza,ani.jrc25@gmail.com,263CMRBQC,2022-02-25,699.52,"BBVA Bancomer, S.A.",2022-02-26 10:43:53,Esperanza Cruz Gutierrez,'012180015421287486,...,TRAD,Reprogramación,699.52,2022-03-01 17:37:10,2022-03-01,3.00,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,ERROR_USUAL,NEW
2048206,bab2d44f-5697-4e4a-82d1-84931ae887b1,Miabi,pablotas_2326@hotmail.com,26MN3298P,2022-02-26,234.77,"Banco Azteca, S.A.",2022-02-26 10:57:34,Genaro Hernández Diaz,'127180013201339289,...,TRAD,Pago del día,234.77,2022-03-01 17:37:13,2022-03-01,3.00,ERROR,DEV.SPEICUENTA INEXISTENTE,ERROR_USUAL,NEW
2048407,2a2a0451-79b4-4d5b-bc0b-b7f395f24e61,Dogos Meños,efrenguapo97@gmail.com,28CN4TGQZ,2022-02-26,866.26,BBVA BANCOMER,2022-02-26 10:40:02,Efren Jair Garcia Hernandez,'012180015242510400,...,TRAD,Pago del día,866.26,2022-03-01 17:37:07,2022-03-01,3.00,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,ERROR_USUAL,NEW
2049061,1df4f2a5-033f-4dee-9847-a2fb36275e3a,Twinning bakery,twinningbrownies@gmail.com,2GJCGNHKM,2022-02-26,134.15,"BBVA Bancomer, S.A.",2022-02-26 10:19:56,Ingrid leon bernal,'012180015882122397,...,TRAD,Pago del día,134.15,2022-03-01 17:37:08,2022-03-01,3.00,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,ERROR_USUAL,NEW


In [32]:
cruce_1_final.shape

(6293, 25)

In [33]:
duplicates_cruce_1_final = cruce_1_final[cruce_1_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_final.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [34]:
duplicates_cruce_1_final.shape

(0, 25)

# CRUCE 2 MES ACTUAL

In [35]:
cruce_2 = query_error_current_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_2['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_2.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,65839,a890a2d0-14d5-4e57-a0cd-0a8616c3dd76,NEPTUNO,SPQYRVHQ5,2022-03-01,128.41,ERROR,Bank account information not available,2022-03-01 08:19:35,3,2022,NEW,SIN_CLABE,a890a2d0-14d5-4e57-a0cd-0a8616c3dd76,2022-03-01,1,Actual,Not found,NaN
1,65314,9eca2c9e-7586-4416-936f-75a768faff41,Cremeria Lety,SBWXML7F8,2022-02-27,64.20,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-03-01 17:37:30,3,2022,NEW,ERROR_USUAL,9eca2c9e-7586-4416-936f-75a768faff41,2022-03-01,1,Actual,TRAD,Pago del día
2,65319,3eb75ab3-5471-42d9-b72b-0829d2073bf5,eltrebol dulcerias,MZ367L2W7,2022-02-27,571.57,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2022-03-01 17:37:23,3,2022,NEW,ERROR_USUAL,3eb75ab3-5471-42d9-b72b-0829d2073bf5,2022-03-01,1,Actual,TRAD,Pago del día
3,65321,2dbdd79f-d96d-4a1d-bcdd-c4bf947eb483,Barbacoa Cathy,M78YS9K3X,2022-02-27,292.27,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2022-03-01 17:37:26,3,2022,NEW,ERROR_USUAL,2dbdd79f-d96d-4a1d-bcdd-c4bf947eb483,2022-03-01,1,Actual,TRAD,Pago del día
4,65323,f71d063f-0442-46ce-ac47-76cf1ea70cd4,MENU FELIZ,NT6RSVPSQ,2022-02-27,124.57,ERROR,DEV.SPEIFALTA INFORMAC PARA CO,2022-03-01 17:37:24,3,2022,NEW,ERROR_USUAL,f71d063f-0442-46ce-ac47-76cf1ea70cd4,2022-03-01,1,Actual,TRAD,Pago del día


In [36]:
cruce_2.shape

(12758, 19)

In [37]:
cruce_2_final = cruce_2[(cruce_2['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
1,65314,9eca2c9e-7586-4416-936f-75a768faff41,Cremeria Lety,SBWXML7F8,2022-02-27,64.20,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-03-01 17:37:30,3,2022,NEW,ERROR_USUAL,9eca2c9e-7586-4416-936f-75a768faff41,2022-03-01,1,Actual,TRAD,Pago del día
2,65319,3eb75ab3-5471-42d9-b72b-0829d2073bf5,eltrebol dulcerias,MZ367L2W7,2022-02-27,571.57,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2022-03-01 17:37:23,3,2022,NEW,ERROR_USUAL,3eb75ab3-5471-42d9-b72b-0829d2073bf5,2022-03-01,1,Actual,TRAD,Pago del día
3,65321,2dbdd79f-d96d-4a1d-bcdd-c4bf947eb483,Barbacoa Cathy,M78YS9K3X,2022-02-27,292.27,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2022-03-01 17:37:26,3,2022,NEW,ERROR_USUAL,2dbdd79f-d96d-4a1d-bcdd-c4bf947eb483,2022-03-01,1,Actual,TRAD,Pago del día
4,65323,f71d063f-0442-46ce-ac47-76cf1ea70cd4,MENU FELIZ,NT6RSVPSQ,2022-02-27,124.57,ERROR,DEV.SPEIFALTA INFORMAC PARA CO,2022-03-01 17:37:24,3,2022,NEW,ERROR_USUAL,f71d063f-0442-46ce-ac47-76cf1ea70cd4,2022-03-01,1,Actual,TRAD,Pago del día
5,65324,8cfc1850-8444-4dea-95cb-3c0a4d15e97c,CARNITAS MORA,VMP2772BT,2022-02-27,337.31,ERROR,DEV.SPEICUENTA INEXISTENTE,2022-03-01 17:37:21,3,2022,NEW,ERROR_USUAL,8cfc1850-8444-4dea-95cb-3c0a4d15e97c,2022-03-01,1,Actual,TRAD,Pago del día


In [38]:
cruce_2_final.shape

(7185, 19)

In [39]:
duplicates_cruce_2_final = cruce_2_final[cruce_2_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
510,64675,0e868049-3ea7-4366-87d4-765e417a07c1,Farmacia Esperanza,263CMRBQC,2022-02-25,699.52,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2022-03-01 17:37:10,3,2022,NEW,ERROR_USUAL,0e868049-3ea7-4366-87d4-765e417a07c1,2022-03-01,1,Actual,TRAD,Pago del día
511,64675,0e868049-3ea7-4366-87d4-765e417a07c1,Farmacia Esperanza,263CMRBQC,2022-02-25,699.52,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2022-03-01 17:37:10,3,2022,NEW,ERROR_USUAL,0e868049-3ea7-4366-87d4-765e417a07c1,2022-03-01,1,Actual,TRAD,Reprogramación
512,64655,79e37950-7df2-4f88-ae28-2e1c7fc57559,LagunaGorda,HBRTZR7L2,2022-02-25,268.31,ERROR,DEV.SPEIFALTA INFORMAC PARA CO,2022-03-01 17:37:07,3,2022,NEW,ERROR_USUAL,79e37950-7df2-4f88-ae28-2e1c7fc57559,2022-03-01,1,Actual,TRAD,Pago del día
513,64655,79e37950-7df2-4f88-ae28-2e1c7fc57559,LagunaGorda,HBRTZR7L2,2022-02-25,268.31,ERROR,DEV.SPEIFALTA INFORMAC PARA CO,2022-03-01 17:37:07,3,2022,NEW,ERROR_USUAL,79e37950-7df2-4f88-ae28-2e1c7fc57559,2022-03-01,1,Actual,TRAD,Reprogramación
514,64524,f5ab724b-152e-4458-b6a9-6839052a4ace,Caribbean Queen,F6M9NM7GV,2022-02-25,"2,290.19",ERROR,DEV.SPEIFALTA INFORMAC PARA CO,2022-03-01 17:37:15,3,2022,NEW,ERROR_USUAL,f5ab724b-152e-4458-b6a9-6839052a4ace,2022-03-01,1,Actual,TRAD,Pago del día


In [40]:
duplicates_cruce_2_final.shape

(1451, 19)

# AGRUPADO POR TIPO DE PAGO - MES ACTUAL

In [41]:
group_tipo_pago = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago.fillna(0,inplace=True)
group_tipo_pago.head()

ultimo_mov_date_only       amount_x                 \
esquema_query                           ERROR_USUAL                  
status_cubeta                                 ERROR                  
tipo_dispersion                                  FP           TRAD   
tipo_pago                            Reprogramación Reprogramación   
0                         2022-03-01           0.00      11,288.07   
1                         2022-03-02           0.00      49,856.29   
2                         2022-03-03           0.00       1,041.08   
3                         2022-03-04           0.00           0.00   
4                         2022-03-05           0.00           0.48   

                                                                         \
esquema_query                                                             
status_cubeta            NEW                                              
tipo_dispersion           FP                        TRAD                  
tipo_pago       Pago del día Reprogramación Pago del día Reprogramación   
0                       0.00           0.00   409,027.55      41,919.46   
1                       0.00           0.00    24,288.67      25,839.81   
2                       0.00           0.00    48,572.11       3,966.13   
3                       0.00           0.00     2,909.09           0.00   
4                       0.00           0.00    59,086.26           0.01   

                            
esquema_query          All  
status_cubeta               
tipo_dispersion             
tipo_pago                   
0               462,235.08  
1                99,984.77  
2                53,579.32  
3                 2,909.09  
4                59,086.75

In [42]:
group_tipo_pago.shape

(32, 8)

In [43]:
group_esquema = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

ultimo_mov_date_only       amount_x                              \
esquema_query                         ERROR_USUAL                               
status_cubeta                               ERROR          NEW                  
tipo_pago                          Reprogramación Pago del día Reprogramación   
0                       2022-03-01      11,288.07   409,027.55      41,919.46   
1                       2022-03-02      49,856.29    24,288.67      25,839.81   
2                       2022-03-03       1,041.08    48,572.11       3,966.13   
3                       2022-03-04           0.00     2,909.09           0.00   
4                       2022-03-05           0.48    59,086.26           0.01   

                          
esquema_query        All  
status_cubeta             
tipo_pago                 
0             462,235.08  
1              99,984.77  
2              53,579.32  
3               2,909.09  
4              59,086.75

In [44]:
group_esquema.shape

(32, 5)

# CRUCE 3 - MES ANTERIOR

In [45]:
cruce_3 = allfiles.merge(query_new_last_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status',
                                               'error_description','esquema_query','status_cubeta']],
                         on='report_id',how='left')
cruce_3['month'].fillna('Not found',inplace=True)
cruce_3.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,44c18ed3-a539-4b92-a820-b8bfe240a2ec,Butic Dani,guillerminatrejoponce@gmail.com,2226SCJM5,2022-01-18,3.20,"Banco Nacional de México, S.A.",2022-02-01 08:16:00,Guillermina Trejo Ponce,'002180904725962832,...,TRAD,Reprogramación,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,eca46980-db16-48bb-9524-2590bffe7abc,Delicarne,juanclusa@gmail.com,2228H2KBV,2022-02-01,284.09,"Banco Azteca, S.A.",2022-02-01 08:15:30,Ursula Pacheco Gomez,'127180013357157409,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,a7b19a9a-e103-415e-95a7-e8b2d13a1b0e,Pintura Y Barnices La Brocha,noelrosalesuscanga@hotmail.com,2228YHDPP,2022-02-01,"2,029.10","HSBC México, S.A.",2022-02-01 08:14:58,Hilda luisa Uscanga Hernández,'021847040592603722,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,38d51729-0d41-4123-9918-ae8d4d65432f,La Campesina,la_campesina2020@outlook.es,222G5TLRQ,2022-02-01,302.14,BBVA BANCOMER,2022-02-01 08:14:58,Jose Luis Contreras Armenta,'012221001867460390,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,3a67c0d6-90ef-4d85-a491-048f02783655,Farmacia Marella,dianarm94@hotmail.com,222PPYCK4,2022-02-01,"2,006.07","Banco Azteca, S.A.",2022-02-01 08:16:32,Diana Laura Ramirez Martinez,'127622013434344716,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [46]:
cruce_3.shape

(4949950, 25)

In [47]:
cruce_3_final = cruce_3[(cruce_3['month'] != 'Not found')]
cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_3_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_2552\2713612933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
8795,ae868353-ad64-43b7-aeb2-8a2058b123bf,Calzados Viany,moralesviany742@gmail.com,58DSB8D6L,2021-12-07,1.80,"Banco Compartamos, S.A.",2022-02-01 08:16:00,Gloria Bautista Hernandez,'130180001616945416,...,TRAD,Reprogramación,1.80,2022-02-01 13:23:55,2022-02-01,2.00,ERROR,Tipo de operacion erronea,ERROR_USUAL,NEW


In [48]:
cruce_3_final.shape

(1, 25)

In [49]:
duplicates_cruce_3_final = cruce_3_final[cruce_3_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_3_final.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [50]:
duplicates_cruce_3_final.shape

(0, 25)

# CRUCE 4 - MES ANTERIOR

In [51]:
cruce_4 = query_new_last_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_4['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_4.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,48686,ae868353-ad64-43b7-aeb2-8a2058b123bf,Calzados Viany,58DSB8D6L,2021-12-07,1.80,ERROR,Tipo de operacion erronea,2022-02-01 13:23:55,2,2022,NEW,ERROR_USUAL,ae868353-ad64-43b7-aeb2-8a2058b123bf,2022-02-01,1,NaN,TRAD,Reprogramación


In [52]:
cruce_4.shape

(1, 19)

In [53]:
cruce_4_final = cruce_4[(cruce_4['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,48686,ae868353-ad64-43b7-aeb2-8a2058b123bf,Calzados Viany,58DSB8D6L,2021-12-07,1.80,ERROR,Tipo de operacion erronea,2022-02-01 13:23:55,2,2022,NEW,ERROR_USUAL,ae868353-ad64-43b7-aeb2-8a2058b123bf,2022-02-01,1,NaN,TRAD,Reprogramación


In [54]:
cruce_4_final.shape

(1, 19)

In [55]:
duplicates_cruce_4_final = cruce_4_final[cruce_4_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago


In [56]:
duplicates_cruce_4_final.shape

(0, 19)

# AGRUPADO POR TIPO DE PAGO - MES ANTERIOR

In [57]:
group_tipo_pago_last_month = pd.pivot_table(cruce_3_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago_last_month.fillna(0,inplace=True)
group_tipo_pago_last_month.head()

ultimo_mov_date_only       amount_x     
esquema_query                           ERROR_USUAL  All
status_cubeta                                   NEW     
tipo_dispersion                                TRAD     
tipo_pago                            Reprogramación     
0                         2022-02-01           1.80 1.80
1                                All           1.80 1.80

In [58]:
group_tipo_pago_last_month.shape

(2, 3)

In [59]:
group_esquema_last_month = pd.pivot_table(cruce_4_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema_last_month.fillna(0,inplace=True)
group_esquema_last_month.head()

ultimo_mov_date_only         amount     
esquema_query                         ERROR_USUAL  All
status_cubeta                                 NEW     
tipo_pago                          Reprogramación     
0                       2022-02-01           1.80 1.80
1                              All           1.80 1.80

In [60]:
group_esquema_last_month.shape

(2, 3)

In [61]:
writer = pd.ExcelWriter('FASE V. DATA POLIZA ERROR-USUAL MARZO 2022.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_tipo_pago.to_excel(writer, sheet_name='GRUPO TIPO PAGO')
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
group_tipo_pago_last_month.to_excel(writer, sheet_name='GRUPO TIPO PAGO-MESES ANT')
group_esquema_last_month.to_excel(writer, sheet_name='GRUPO ESQUEMA-MESES ANT')
cruce_1_final.to_excel(writer, sheet_name='DETALLE',index=False)
cruce_3_final.to_excel(writer, sheet_name='DETALLE-MESES ANTERIORES',index=False)
writer.close()